![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [1]:
import pandas as pd
import numpy as np



In [2]:
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64
mortgage
--------------
yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64
previous_outcome
--------------
nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64
campaign_outcome
--------------
no     36548
yes     4640
Name: campaign_outcome, dtype: int64


In [5]:
# create client df with only columns 0-7
client_df = df.iloc[:, 0:7]
#print(client_df.head())

   client_id  age        job  marital    education credit_default mortgage
0          0   56  housemaid  married     basic.4y             no       no
1          1   57   services  married  high.school        unknown       no
2          2   37   services  married  high.school             no      yes
3          3   40     admin.  married     basic.6y             no       no
4          4   56   services  married  high.school             no       no


In [8]:
# replace . with _ in job and educatin columns
client_df["job"] = client_df["job"].str.replace(".", "_", regex=False)
client_df["education"] = client_df["education"].str.replace(".", "_", regex=False)

print(client_df.head())

   client_id  age        job  marital    education credit_default mortgage
0          0   56  housemaid  married     basic_4y             no       no
1          1   57   services  married  high_school        unknown       no
2          2   37   services  married  high_school             no      yes
3          3   40     admin_  married     basic_6y             no       no
4          4   56   services  married  high_school             no       no


In [10]:
# change "unknown" to np.nan in client_df
client_df = client_df.replace("unknown", np.nan)

# print the number of missing values in each column
#print(client_df.isnull().sum())

In [13]:
# convert credit_default and mortgate to 0 and 1

# print unique values in credit_default and mortgage
#print(client_df["credit_default"].unique())
#print(client_df["mortgage"].unique())

# convert credit_default and mortgage 1 if yes, else 0
client_df["credit_default"] = client_df["credit_default"].apply(lambda x: 1 if x == "yes" else 0)
client_df["mortgage"] = client_df["mortgage"].apply(lambda x: 1 if x == "yes" else 0)

# change the data type of credit_default and mortgage to bool
client_df["credit_default"] = client_df["credit_default"].astype(bool)
client_df["mortgage"] = client_df["mortgage"].astype(bool)

print(client_df.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   client_id       41188 non-null  int64 
 1   age             41188 non-null  int64 
 2   job             40858 non-null  object
 3   marital         41108 non-null  object
 4   education       39457 non-null  object
 5   credit_default  41188 non-null  bool  
 6   mortgage        41188 non-null  bool  
dtypes: bool(2), int64(2), object(3)
memory usage: 1.6+ MB
None


In [14]:
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   age                         41188 non-null  int64  
 2   job                         41188 non-null  object 
 3   marital                     41188 non-null  object 
 4   education                   41188 non-null  object 
 5   credit_default              41188 non-null  object 
 6   mortgage                    41188 non-null  object 
 7   month                       41188 non-null  object 
 8   day                         41188 non-null  int64  
 9   contact_duration            41188 non-null  int64  
 10  number_contacts             41188 non-null  int64  
 11  previous_campaign_contacts  41188 non-null  int64  
 12  previous_outcome            41188 non-null  object 
 13  cons_price_idx              411

In [21]:
# create campaign_df with columns 0, 10, 9, 11, 12, 15 and month, day
campaign_df = df.iloc[:, [0, 10, 9, 11, 12, 15, 7, 8]]
#print(campaign_df.head())
print(campaign_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   client_id                   41188 non-null  int64 
 1   number_contacts             41188 non-null  int64 
 2   contact_duration            41188 non-null  int64 
 3   previous_campaign_contacts  41188 non-null  int64 
 4   previous_outcome            41188 non-null  object
 5   campaign_outcome            41188 non-null  object
 6   month                       41188 non-null  object
 7   day                         41188 non-null  int64 
dtypes: int64(5), object(3)
memory usage: 2.5+ MB
None


In [22]:
# create a copy of campaign_df to avoid SettingWithCopyWarning
campaign_df = campaign_df.copy()

# convert previous_outcome to 0 and 1, 1 if 'success', else 0
campaign_df["previous_outcome"] = campaign_df["previous_outcome"].apply(lambda x: 1 if x == "success" else 0)
# convert campaign_outcome to 0 and 1, 1 if 'yes', else 0
campaign_df["campaign_outcome"] = campaign_df["campaign_outcome"].apply(lambda x: 1 if x == "yes" else 0)
# change the data type of campaign_outcome and previous_outcome to bool
campaign_df["campaign_outcome"] = campaign_df["campaign_outcome"].astype(bool)
campaign_df["previous_outcome"] = campaign_df["previous_outcome"].astype(bool)

print(campaign_df.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   client_id                   41188 non-null  int64 
 1   number_contacts             41188 non-null  int64 
 2   contact_duration            41188 non-null  int64 
 3   previous_campaign_contacts  41188 non-null  int64 
 4   previous_outcome            41188 non-null  bool  
 5   campaign_outcome            41188 non-null  bool  
 6   month                       41188 non-null  object
 7   day                         41188 non-null  int64 
dtypes: bool(2), int64(5), object(1)
memory usage: 2.0+ MB
None


In [23]:
# create a new column 'last_contact_date' by combining 'day' and 'month' with the year 2022
campaign_df['last_contact_date'] = pd.to_datetime(campaign_df['day'].astype(str) + '-' + campaign_df['month'] + '-2022', format='%d-%b-%Y')

# drop the 'month' and 'day' columns
campaign_df = campaign_df.drop(columns=['month', 'day'])

print(campaign_df.head())

   client_id  number_contacts  contact_duration  previous_campaign_contacts  \
0          0                1               261                           0   
1          1                1               149                           0   
2          2                1               226                           0   
3          3                1               151                           0   
4          4                1               307                           0   

   previous_outcome  campaign_outcome last_contact_date  
0             False             False        2022-05-13  
1             False             False        2022-05-19  
2             False             False        2022-05-23  
3             False             False        2022-05-27  
4             False             False        2022-05-03  


In [25]:
# create economics_df from df with col 0, 13, 14 
economics_df = df.iloc[:, [0, 13, 14]]
print(economics_df.head())

   client_id  cons_price_idx  euribor_three_months
0          0          93.994                 4.857
1          1          93.994                 4.857
2          2          93.994                 4.857
3          3          93.994                 4.857
4          4          93.994                 4.857


In [26]:
client_df.to_csv("client.csv", index=False)
campaign_df.to_csv("campaign.csv", index=False)
economics_df.to_csv("economics.csv", index=False)